In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import os
import pandas as pd
import pickle

In [50]:
def img_preproc(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Only look at white parts
    _,bwimg=cv2.threshold(gray,240,255,cv2.THRESH_BINARY)
    return bwimg

def get_chars(img):
    """Break an image of numbers into a list of images of characters
    
    Reference: https://stackoverflow.com/questions/50713023/license-plate-character-segmentation-python-opencv
    """
    chars=[]
    _,contours,_=cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    # Sort bounding boxes from left to right
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
    for ctr in contours:
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr)
        # Getting ROI
        roi = img[y:y+h, x:x+w]
        chars.append(roi)
    return chars

def filter_chars(img):
    """Filter out non-character glyphs based on height/width ratio
    """
    h,w=img.shape
    ratio=h/w
    if ratio<=1 or ratio>2.1:
        return False
    else:
        return True

def conform_size(img):
    """Resize images to conform to the size of MNIST images, which is 28x28
    """
    target_width=20
    target_height=20
    padding_second=4
    h,w=img.shape
    if w>h:
        # Resize without stretching
        new_h=int(np.round(h*target_height/w))
        resized_img = cv2.resize(img, (target_width, new_h))
        # Pad image
        blank=target_height-new_h
        pad_t=int(np.floor(blank/2))
        pad_b=int(np.floor((blank+1)/2))
        padded_img=cv2.copyMakeBorder(resized_img,pad_t,pad_b,0,0,cv2.BORDER_CONSTANT,value=0)
        # Do padding again to add blanks so that the image resembles those from MNIST
        # Otherwise the trained model is destined to fail
        padded_img2=cv2.copyMakeBorder(padded_img,padding_second,padding_second,padding_second,padding_second,cv2.BORDER_CONSTANT,value=0)
    else:
        new_w=int(np.round(w*target_width/h))
        resized_img = cv2.resize(img, (new_w,target_height))
        blank=target_width-new_w
        pad_l=int(np.floor(blank/2))
        pad_r=int(np.floor((blank+1)/2))
        padded_img=cv2.copyMakeBorder(resized_img,0,0,pad_l,pad_r,cv2.BORDER_CONSTANT,value=0)
        padded_img2=cv2.copyMakeBorder(padded_img,padding_second,padding_second,padding_second,padding_second,cv2.BORDER_CONSTANT,value=0)
    # 0-255 to 0-1
    normalized=padded_img2/255
    normalized = normalized.reshape(28, 28, 1)
    return normalized

In [51]:
path_train_imgs=glob.glob("training/*")

In [52]:
train_x=[]
train_y=[]

In [53]:
for path in path_train_imgs:
    label=os.path.basename(path).split(".")[0]
    labels=[int(x) for x in list(label)]
    img_orig=cv2.imread(path)
    img=img_preproc(img_orig)
    characters=get_chars(img)
    filt_chars=[img for img in characters if filter_chars(img)]
    # "Normalized" characters
    norm_chars=[conform_size(char) for char in filt_chars]
    train_x=train_x+norm_chars
    train_y=train_y+labels

In [54]:
train_y=np.array(train_y)
train_x=np.array(train_x)

In [55]:
data=(train_x,train_y)

In [56]:
with open("data.pkl","wb") as f:
    pickle.dump(data,f)

=== Data generation ends here ===

In [ ]:
for i in range(path_train_imgs):
    char=train_x[i,:,:,:]